<a href="https://colab.research.google.com/github/kabanoki/WebPageToChatQA/blob/main/WebPageToChatQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **■■ChatGPT API■■**

# !!!!!上から一つづつ[▶]を起動していってください。!!!!!

---

In [ ]:
!pip install langchain==0.0.125 openai==0.27.2 chromadb==0.3.14 pypdf==3.7.0 tiktoken==0.3.3 gradio==3.23 unstructured

---

# *API設定：*

最大20ドルまで使用可能です。

おそらく一回の検索で0.035ドルぐらいです。

GPT-4はまだ使えません。

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""#@param {type:"string"}
model = 'gpt-3.5-turbo' #@param ["gpt-3.5-turbo", "gpt-4"] {allow-input: true}


---

# *データソースURL：*

URLの内容を元に質問できます。

※URLを切り替えたら更新してください。

※https:// から入力してください。

※複数URLを設定する場合は[,]で区切ってください。（スペースなし）

例
https://www.japanprint.co.jp/,https://www.japanprint.co.jp/strength.html,https://www.japanprint.co.jp/company.html, https://www.japanprint.co.jp/location.html,https://pzgd6tejl.jbplt.jp/,

In [ ]:
url = ""#@param {type:"string"}

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
import openai
from langchain import text_splitter
from langchain.document_loaders.url import UnstructuredURLLoader
from langchain import PromptTemplate


urls = url.split(",")

print(urls)

loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 0,
    length_function = len,
)

texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name=model),
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_kwargs={"k": 1})
)

template = """
あなたは親切なアシスタントです。下記の質問に日本語で回答してください。
質問：{question}
回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)



---

# *質問：*

「新インターフェース」を起動してください

In [ ]:
#@title  **新インターフェース**
import gradio as gr

def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):
    query = history[-1][0]
    query = prompt.format(question=query)
    answer = qa.run(query)
    history[-1][1] = answer
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=350)

    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
                show_label=False,
                placeholder="入力後にエンターキーを押してください。",
            ).style(container=False)

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch()

In [ ]:
#@title 旧インターフェース
query = "\u5927\u962A\u652F\u90E8\u306E\u63A1\u7528\u6642\u306E\u6708\u7D66\u3092\u6559\u3048\u3066\u304F\u3060\u3055\u3044\u3002"#@param {type:"string"}
query = prompt.format(question=query)
answer = qa.run(query)
print(answer)